# **응급상황 자동 인식 및 응급실 연계 서비스**
# **단계4 : 통합-모듈화**

## **0.미션**

단계 4에서는, 단계1,2,3 에서 생성한 함수들을 모듈화하고, 단위 테스트 및 파이프라인 코드를 작성합니다.

* **미션6**
    * Python 코드 모듈화
        * 각 모듈 코드 및 모델, 데이터파일을 일관성 있게 정리
        * .py 파일 생성 ==> 라이브러리 로딩, 각 task를 위한 함수 생성


## **1.환경설정**

* 경로 설정

구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/project6_2/'

In [ ]:
!pip install -r /content/drive/MyDrive/project6_2/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


## 2.모듈 구성하기

In [ ]:
%%writefile /content/drive/MyDrive/project6_2/emergency.py

import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import openai
from openai import OpenAI
import json
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import load_dataset, Dataset

from sklearn.model_selection import train_test_split
from sklearn.metrics import *

from haversine import haversine

from warnings import filterwarnings
filterwarnings('ignore')

# 0. load key file------------------

def register_key(key):
  os.environ['OPENAI_API_KEY'] = key

# 1-1 audio2text--------------------

def audio_to_text(audio_path, filename):
    # OpenAI 클라이언트 생성
    client = OpenAI()

    # 오디오 파일을 읽어서, 위스퍼를 사용한 변환
    filename = filename
    audio_file = open(audio_path + filename, "rb")
    transcript = client.audio.transcriptions.create(
        file=audio_file,
        model="whisper-1",
        language="ko",
        response_format="text",
    )

    # 결과 반환
    return transcript

# 1-2 text2summary------------------

def text_summary(input_text):
    # OpenAI 클라이언트 생성
    client = OpenAI()

    # 시스템 역할과 응답 형식 지정
    system_role = '''
                   너는 전화에서 핵심을 요약하는 어시스턴트야.
                   횡설수설하는 전화내용에서 핵심 키워드만 추려내야 해.
                   그외에는 내용에 포함해선 안돼
                   응답은 다음의 형식을 지켜줘.
                   {"summary": \"텍스트 요약\"}.
                   '''

    # 입력데이터를 GPT-3.5-turbo에 전달하고 답변 받아오기
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": system_role
            },
            {
                "role": "user",
                "content": input_text
            }
        ]
    )

    # 응답형식을 정리하고 return
    return response.choices[0].message.content

# 2. model prediction------------------

def predict(text, model, tokenizer):
    # 모델이 있는 디바이스 확인
    device = next(model.parameters()).device

    # 입력 문장 토크나이징
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # 각 텐서를 모델과 같은 디바이스로 이동
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # 모델 예측
    with torch.no_grad():
        outputs = model(**inputs)

    # 로짓을 소프트맥스로 변환하여 확률 계산
    logits = outputs.logits
    probabilities = logits.softmax(dim=1)

    # 가장 높은 확률을 가진 클래스 선택
    pred = torch.argmax(probabilities, dim=-1).item()

    return pred+1, probabilities


# 3-1. get_distance------------------

def get_distance(start_lat, start_lng, dest_lat, dest_lng, c_id, c_key):

    url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": c_id,
        "X-NCP-APIGW-API-KEY": c_key,
    }
    params = {
        "start": f"{start_lng},{start_lat}",  # 출발지 (경도, 위도)
        "goal": f"{dest_lng},{dest_lat}",    # 목적지 (경도, 위도)
        "option": "trafast"  # 실시간 빠른 길 옵션
    }

    # 요청하고, 답변 받아오기
    response = requests.get(url=url, headers=headers, params=params)
    flag = True

    if response.status_code == 200 :
      data = response.json()
      code = data['code']
      if code == 0 :
        pass
      elif code == 1 :
        print("요청 처리 실패. 출발지와 도착지가 동일함")
        flag = False
      elif code == 2 :
        print("요청 처리 실패. 출발지 또는 도착지가 도로 주변이 아님")
        flag = False
      elif code == 3 :
        print("요청 처리 실패. 자동차 길 찾기 결과 제공 불가")
        flag = False
      elif code == 4 :
        print("요청 처리 실패. 경유지가 도로 주변이 아님")
        flag = False
      else :
        print("	요청 처리 실패. 경유지를 포함한 직선거리 합이 1500 km 이상인 경로가 요청됨")
        flag = False
    else:
      print("Error:", response.status_code)
      flag = False


    if flag == False :
      dist = 1000000007
    else :
      dist = data['route']['trafast'][0]['summary']['distance']  # m(미터)

    return dist / 1000

# 3-2. get_haversine------------------

def get_haversine(y, x, ty, tx):
    return haversine((y, x), (ty, tx), unit='km')

# 3-3. recommendation------------------

def emergency_recommendation(y, x,  emergency, c_id, c_key) :
  dist_list = []
  near = {}

  ax, ay = 0.3, 0.3
  sx, sy, ex, ey = x - ax, y - ay, x + ax, y + ay

  for i in range(len(emergency)):
      name = emergency.loc[i, 'name']
      target_x, target_y = emergency.loc[i, '경도'], emergency.loc[i, '위도']
      if target_y < sy or target_y > ey or target_x < sx or target_x > ex:
          continue
      near[name] = get_haversine(y, x, target_y, target_x)


  near = sorted(near.items(), key=lambda x: x[1])[:10]
  near_hospital = emergency[emergency['name'].isin([hospital[0] for hospital in near])]
  near_hospital = near_hospital.reset_index(drop=True)

  for i in range(len(near_hospital)):
      name = near_hospital.loc[i, 'name']
      tel = near_hospital.loc[i, 'tel']
      addr = near_hospital.loc[i, 'addr']
      target_x, target_y = near_hospital.loc[i, '경도'], near_hospital.loc[i, '위도']
      dist = get_distance(y, x, target_y, target_x, c_id, c_key)
      dist_list.append((name, tel, addr, dist))

  dist_list = sorted(dist_list, key=lambda x: x[3])[:3]

  return dist_list

Overwriting /content/drive/MyDrive/kt_aivle/6-2차_미니_프로젝트/emergency.py
